In [7]:
import sys
sys.path.append('/home/xhh/notebooks/GNN/pytorch-template')
from src.datamodules.components.data import get_data
import torch

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

data, num_features, num_classes, processed_dir = get_data('pubmed', split='full')

# data.to(device)

edge_index = data.edge_index
num_nodes = data.num_nodes

data

Data(x=[19717, 500], edge_index=[2, 88648], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])

In [13]:
data.x.device

device(type='cpu')

In [ ]:
import copy

data2 = copy.copy(data)
data2.to(device)

In [450]:
from torch_sparse import SparseTensor

row, col = edge_index.cpu()
adj_t = SparseTensor(
    row=row, col=col, value=torch.arange(col.size(0)),
    sparse_sizes=(data.num_nodes, data.num_nodes)).t()
# adj_t.to(device)

In [443]:
data.edge_index.to('cpu')
data.edge_index

tensor([[    0,     0,     0,  ..., 19714, 19715, 19716],
        [ 1378,  1544,  6092,  ..., 12278,  4284, 16030]], device='cuda:1')

In [72]:
_, degree = torch.unique(row, return_counts=True)
max_deg = degree.max().item()
max_deg

171

In [66]:
%%time

edge = torch.full((num_nodes+1, max_deg), num_nodes)

for v in range(num_nodes):
    neighbors = adj_t[v, :].coo()[1]  # 31s
    # neighbors = edge_index[1, edge_index[0] == v]  # 315s

    len_neighbors = neighbors.shape[0]
    if len_neighbors == 0:
        edge.data[v, 0] = v
    else:
        edge.data[v, :len_neighbors] = neighbors

edge

CPU times: user 8h 18min 47s, sys: 35.4 s, total: 8h 19min 23s
Wall time: 5min 15s


tensor([[   249,    969,   1408,  ..., 232965, 232965, 232965],
        [  1394,   1426,   8476,  ..., 232965, 232965, 232965],
        [  3077,   4300,   4424,  ..., 232965, 232965, 232965],
        ...,
        [  3616,   4859,   6627,  ..., 232965, 232965, 232965],
        [  1304,   2311,   4883,  ..., 232965, 232965, 232965],
        [232965, 232965, 232965,  ..., 232965, 232965, 232965]])

In [80]:
adj_t = adj_t.to(device)
v = torch.tensor([2943,  8359, 10199, 10471, 11485, 15572,  8249])
# adj_t[v, :].coo()[1]
row, col, layer_e = adj_t[v, :].coo()
adj_t[v, :]

SparseTensor(row=tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                           1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4,
                           4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
                           5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
                           6, 6, 6, 6, 6], device='cuda:1'),
             col=tensor([    1,  2070,  2461,  4416,  5519,  5966,  6222,  9412,  9509, 13400,
                           14746, 17366,     1,  1161,  1304,  2461,  4249,  6679,  8317, 10051,
                           10140, 10487, 11197, 11631, 11880, 12230, 13705, 14055, 14877, 17366,
                           18540,     1, 14055, 14746, 18681,     2,   569,  3227,  3537,  3634,
                            4996,  5238,  5870,  6718,  9965, 14586,     2,  2218,  2519,  3978,
                            5150,  6495,  6659, 12076, 12241, 

In [129]:
col, col_inv = col.unique(return_inverse=True)

In [173]:
%%time
idxs, invs, ptrs, inc = [], [], [], []
for bn in range(1000000):
    idxs.append(1)
    invs.append(1)
    ptrs.append(1)
    inc.append(1)

CPU times: user 333 ms, sys: 8 ms, total: 341 ms
Wall time: 340 ms


In [180]:
%%time
num = 20000
idxs = [torch.tensor([])] * num
# idxs = [None] * num
# idxs = torch.empty(num)
# idxs = torch.zeros(num).to(device)
for bn in range(num):
    idxs[bn] = 1


# torch.tensor(idxs).to(device)

CPU times: user 65.5 ms, sys: 0 ns, total: 65.5 ms
Wall time: 65.1 ms


In [441]:
%%time

num = 100000
for bn in range(num):
    torch.ones(num, device=device)

CPU times: user 881 ms, sys: 0 ns, total: 881 ms
Wall time: 891 ms


In [ ]:
from torch_sparse import matmul

x_v = data.x[v].to(device)

matmul(adj_t[v, :].t(), x_v)[col].shape

In [213]:
from torch_scatter import scatter

h_u = scatter(x_v[row], col_inv, dim=0, reduce='sum') + data.x[col].to(device)
h_u.shape

torch.Size([93, 500])

---

In [14]:
data

Data(x=[19717, 500], edge_index=[2, 88648], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])

In [16]:
data.val_mask.sum()

tensor(500)

In [25]:
from src.utils.index import mask_to_index
from torch_geometric.utils import subgraph

mask_to_index(subgraph(data.val_mask, data.edge_index, return_edge_mask=True)[2])

tensor([ 297,  322,  323,  377,  443,  498,  507,  553,  642,  687,  757,  772,
         830,  869,  870,  885,  974, 1010, 1011, 1030, 1031, 1098, 1175, 1199,
        1303, 1342, 1425, 1521, 1545, 1546, 1547, 1739, 1800, 1825, 1840, 1858,
        1916, 1989, 1990, 2014, 2059, 2074, 2145, 2158, 2159, 2178, 2291, 2320,
        2321, 2322, 2323, 2324, 2350, 2401])

In [26]:
subgraph(data.val_mask, data.edge_index, relabel_nodes=True)

(tensor([[  0,   1,   2,  17,  39,  46,  50,  56,  72,  84,  95, 101, 125, 136,
          136, 140, 158, 176, 176, 181, 182, 198, 222, 224, 241, 251, 264, 288,
          297, 297, 297, 324, 341, 347, 350, 351, 369, 383, 383, 385, 399, 402,
          423, 428, 428, 429, 458, 468, 468, 468, 468, 468, 469, 473],
         [ 84, 383, 402, 458, 297,  72, 176, 423,  46,   0, 469, 136, 297, 101,
          264, 222, 251,  50, 468, 297, 468, 428, 140, 341, 369, 158, 136, 473,
           39, 125, 181, 385, 224, 351, 468, 347, 241,   1, 399, 324, 383,   2,
           56, 198, 468, 468,  17, 176, 182, 350, 428, 429,  95, 288]]),
 None)

In [29]:
torch.rand((data.val_mask.shape[0], )) * 2 -1

tensor([-0.0935,  0.9052, -0.6369,  ...,  0.3908,  0.7589,  0.5236])